# Deep Convolutional Q-Learning for Pac-Man

## Part 0 - Installing the required packages and importing the libraries

### Installing Gymnasium

In [ ]:
!pip install gymnasium
!pip install "gymnasium[atari, accept-rom-license]"
!apt-get install -y swig
!pip install gymnasium[box2d]

### Importing the libraries

In [ ]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from collections import deque
from torch.utils.data import DataLoader, TensorDataset

## Part 1 - Building the AI

### Creating the architecture of the Neural Network

In [ ]:
class Network(nn.Module):
  def __init__(self, action_size, seed = 42) -> None:
      super(Network, self).__init__()
      self.seed = torch.manual_seed(seed)                                                 #in_channels = number of input dimensions, 3 as we use RGB.
      self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size = 8, stride = 4) #out_channels = number of feature maps.
      self.bn1 = nn.BatchNorm2d(32)
      self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size = 4, stride = 2)
      self.bn2 = nn.BatchNorm2d(64)
      self.conv3 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size = 3, stride = 1)
      self.bn3 = nn.BatchNorm2d(64)
      self.conv4 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size = 3, stride = 1)
      self.bn4 = nn.BatchNorm2d(128)
      self.fc1 = nn.Linear(in_features=(128 * 10 * 10), out_features=512)
      self.fc2 = nn.Linear(in_features=512, out_features=256)
      self.fc3 = nn.Linear(in_features=256, out_features=action_size)

  def forward(self, state):
    x = F.relu(self.bn1(self.conv1(state)))
    x = F.relu(self.bn2(self.conv2(x)))
    x = F.relu(self.bn3(self.conv3(x)))
    x = F.relu(self.bn4(self.conv4(x)))
    x = x.view(x.size(0), -1) #Flatten the tensor
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    return self.fc3(x)



## Part 2 - Training the AI

### Setting up the environment

In [ ]:
import gymnasium as gym
env = gym.make('MsPacmanDeterministic-v0', full_action_space = False)
state_shape = env.observation_space.shape
state_size = state_shape[0]
number_actions = env.action_space.n

/usr/local/lib/python3.10/dist-packages/gymnasium/envs/registration.py:513: DeprecationWarning: WARN: The environment MsPacmanDeterministic-v0 is out of date. You should consider upgrading to version `v4`.
  logger.deprecation(


### Initializing the hyperparameters

In [ ]:
learning_rate = 5e-4
minibatch_size = 64
discount_factor = 0.99

### Preprocessing the frames

In [ ]:
from PIL import Image
from torchvision import transforms

def preprocess_frame(frame):
  frame = Image.fromarray(frame) #images come as a numpy array, we have to make them tensors to fed them into the ANN
  preprocess = transforms.Compose([transforms.Resize((128,128)), transforms.ToTensor()]) #resizing the images to 128px x 128px makes the training easier
  return preprocess(frame).unsqueeze(0)

### Implementing the DCQN class

In [ ]:
class Agent():

  def __init__(self, action_size) -> None:
     self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
     self.action_size = action_size
     self.local_qnetwork = Network(action_size).to(self.device) #The Q-network that will select the actions
     self.target_qnetwork = Network(action_size).to(self.device) #The Q-network that will calculate the target Q-values that will be used in the training of the local Q-network
     self.optimizer = optim.Adam(self.local_qnetwork.parameters(), lr = learning_rate)
     self.memory = deque(maxlen = 10000)

  def step(self, state, action, reward, next_state, done):
    state = preprocess_frame(state)
    next_state = preprocess_frame(next_state)
    self.memory.append((state, action, reward, next_state, done))
    if len(self.memory) > minibatch_size:
      experiences = random.sample(self.memory, minibatch_size)
      self.learn(experiences, discount_factor)

  def act(self, state, epsilon = 0.):
    state = preprocess_frame(state).to(self.device)
    self.local_qnetwork.eval() #set to evaluation mode to get q-values
    with torch.no_grad(): #any gradient computation is disabled, no learning
      action_values = self.local_qnetwork(state)
    self.local_qnetwork.train() #back to training mode
    if random.random() > epsilon: #epsilon greedy selection policy
      return np.argmax(action_values.cpu().data.numpy())
    else:
      return random.choice(np.arange(self.action_size))

  def learn(self, experiences, discount_factor):
    states, actions, rewards, next_states, dones = zip(*experiences)
    states = torch.from_numpy(np.vstack(states)).float().to(self.device)
    actions = torch.from_numpy(np.vstack(actions)).long().to(self.device)
    rewards = torch.from_numpy(np.vstack(rewards)).float().to(self.device)
    next_states = torch.from_numpy(np.vstack(next_states)).float().to(self.device)
    dones = torch.from_numpy(np.vstack(dones).astype(np.uint8)).float().to(self.device)
    next_q_targets = self.target_qnetwork(next_states).detach().max(1)[0].unsqueeze(1)
    q_targets = rewards + discount_factor * next_q_targets * (1 - dones)
    q_expected = self.local_qnetwork(states).gather(1, actions)
    loss = F.mse_loss(q_expected, q_targets)
    self.optimizer.zero_grad()
    loss.backward()
    self.optimizer.step()



### Initializing the DCQN agent

In [ ]:
agent = Agent(action_size=number_actions)

### Training the DCQN agent

In [ ]:
number_episodes = 2000
max_num_timesteps_per_episode = 10000
epsilon_starting_value = 1.0
epsilon_ending_value = 0.01
epsilon_decay_value = 0.995
epsilon = epsilon_starting_value
scores_on_100_episodes = deque(maxlen = 100)

for episode in range(1, number_episodes + 1):
  state, _ = env.reset() #Then env must be reseted at the begining of each new episode. The '_' is to discard the rest of the information returned.
  score = 0 #acumulative reward
  for t in range(max_num_timesteps_per_episode):
    action = agent.act(state, epsilon)
    next_state, reward, done, _, _ = env.step(action)
    agent.step(state, action, reward, next_state, done)
    state = next_state
    score += reward
    if done:
      break
  scores_on_100_episodes.append(score)
  epsilon = max(epsilon_ending_value, epsilon * epsilon_decay_value)
  print('\rEpisode {}\tAverage Score: {:.2f}'.format(episode, np.mean(scores_on_100_episodes)), end="")
  if episode % 100 == 0:
    print('\rEpisode {}\tAverage Score: {:.2f}'.format(episode, np.mean(scores_on_100_episodes)))
  if np.mean(scores_on_100_episodes) >= 500.0:
    print('\rEnvironment solved in {:d} episodes\tAverage Score: {:.2f}'.format(episode - 100, np.mean(scores_on_100_episodes)))
    torch.save(agent.local_qnetwork.state_dict(), 'checkpoint.pth')
    break

Episode 100	Average Score: 289.50
Episode 200	Average Score: 342.30
Episode 300	Average Score: 372.00
Episode 334	Average Score: 366.50

## Part 3 - Visualizing the results

In [ ]:
import glob
import io
import base64
import imageio
from IPython.display import HTML, display
from gym.wrappers.monitoring.video_recorder import VideoRecorder

def show_video_of_model(agent, env_name):
    env = gym.make(env_name, render_mode='rgb_array')
    state, _ = env.reset()
    done = False
    frames = []
    while not done:
        frame = env.render()
        frames.append(frame)
        action = agent.act(state)
        state, reward, done, _, _ = env.step(action)
    env.close()
    imageio.mimsave('video.mp4', frames, fps=30)

show_video_of_model(agent, 'MsPacmanDeterministic-v0')

def show_video():
    mp4list = glob.glob('*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        display(HTML(data='''<video alt="test" autoplay
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
    else:
        print("Could not find video")

show_video()